In [1]:
!rm -rf lightning_logs

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import lightning
import pandas as pd
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics

In [3]:
lightning.seed_everything(0)

Seed set to 0


0

In [4]:
train_df = pd.read_csv("train.csv")

In [5]:
drop_columns = ["PassengerId", "Name", "Ticket", "Cabin"]
train_df = train_df.drop(columns=drop_columns)
train_df = train_df.dropna()
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
889,1,1,male,26.0,0,0,30.0000,C


In [6]:
sex_encoder = sklearn.preprocessing.LabelEncoder()
train_df["Sex"] = sex_encoder.fit_transform(train_df["Sex"])
embarked_encoder = sklearn.preprocessing.LabelEncoder()
train_df["Embarked"] = embarked_encoder.fit_transform(train_df["Embarked"])
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
885,0,3,0,39.0,0,5,29.1250,1
886,0,2,1,27.0,0,0,13.0000,2
887,1,1,0,19.0,0,0,30.0000,2
889,1,1,1,26.0,0,0,30.0000,0


In [7]:
scaler = sklearn.preprocessing.MinMaxScaler()
train_df[["Age", "Fare"]] = scaler.fit_transform(train_df[["Age", "Fare"]])
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,0.271174,1,0,0.014151,2
1,1,1,0,0.472229,1,0,0.139136,0
2,1,3,0,0.321438,0,0,0.015469,2
3,1,1,0,0.434531,1,0,0.103644,2
4,0,3,1,0.434531,0,0,0.015713,2
...,...,...,...,...,...,...,...,...
885,0,3,0,0.484795,0,5,0.056848,1
886,0,2,1,0.334004,0,0,0.025374,2
887,1,1,0,0.233476,0,0,0.058556,2
889,1,1,1,0.321438,0,0,0.058556,0


In [8]:
train_df, val_df = sklearn.model_selection.train_test_split(train_df, train_size=0.8, random_state=0)

In [9]:
X_train = torch.tensor(train_df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]].values, dtype=torch.float32)
X_val = torch.tensor(val_df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]].values, dtype=torch.float32)
y_train = torch.tensor(train_df["Survived"].values, dtype=torch.int64)
y_val = torch.tensor(val_df["Survived"].values, dtype=torch.int64)

In [10]:
X_train.shape

torch.Size([569, 7])

In [11]:
X_val.shape

torch.Size([143, 7])

In [12]:
y_train.shape

torch.Size([569])

In [13]:
y_val.shape

torch.Size([143])

In [14]:
y_train

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [15]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
train_loader = torch.utils.data.DataLoader(train_dataset)
val_loader = torch.utils.data.DataLoader(val_dataset)

In [16]:
class TitanicClassifier(lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._stack = nn.Sequential(
            nn.Linear(7, 16), nn.ReLU(),
            nn.Linear(16, 2), nn.Softmax(dim=1),
        )

    def forward(self, x):
        return self._stack(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def training_step(self, batch, batch_idx):
        loss, acc = self._compute(batch, batch_idx)
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self._compute(batch, batch_idx)
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        return loss

    def test_step(self, batch, batch_idx):
        loss, acc = self._compute(batch, batch_idx)
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        return loss
    
    def _compute(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = torchmetrics.functional.accuracy(y_hat, y, task="multiclass", num_classes=2)
        return (loss, acc)

In [17]:
trainer = lightning.Trainer(callbacks=[lightning.pytorch.callbacks.early_stopping.EarlyStopping(monitor="val_loss", mode="min")])
clf = TitanicClassifier()
trainer.fit(clf, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\loops\utilities.py:72: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.

  | Name   | Type       | Params | Mode 
----------------------------------------------
0 | _stack | Sequential | 162    | t

Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

In [19]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

In [24]:
test_df = pd.read_csv("test.csv")

In [25]:
drop_columns = ["PassengerId", "Name", "Ticket", "Cabin"]
test_df = test_df.drop(columns=drop_columns)
test_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500,S
414,1,female,39.0,0,0,108.9000,C
415,3,male,38.5,0,0,7.2500,S
416,3,male,NaN,0,0,8.0500,S


In [26]:
test_df[["Age", "Fare"]] = scaler.transform(test_df[["Age", "Fare"]])
test_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,0.428248,0,0,0.015282,Q
1,3,female,0.585323,1,0,0.013663,S
2,2,male,0.773813,0,0,0.018909,Q
3,3,male,0.334004,0,0,0.016908,S
4,3,female,0.271174,1,1,0.023984,S
...,...,...,...,...,...,...,...
413,3,male,NaN,0,0,0.015713,S
414,1,female,0.484795,0,0,0.212559,C
415,3,male,0.478512,0,0,0.014151,S
416,3,male,NaN,0,0,0.015713,S


In [27]:
test_df["Sex"] = sex_encoder.fit_transform(test_df["Sex"])
test_df["Embarked"] = embarked_encoder.fit_transform(test_df["Embarked"])
test_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,0.428248,0,0,0.015282,1
1,3,0,0.585323,1,0,0.013663,2
2,2,1,0.773813,0,0,0.018909,1
3,3,1,0.334004,0,0,0.016908,2
4,3,0,0.271174,1,1,0.023984,2
...,...,...,...,...,...,...,...
413,3,1,NaN,0,0,0.015713,2
414,1,0,0.484795,0,0,0.212559,0
415,3,1,0.478512,0,0,0.014151,2
416,3,1,NaN,0,0,0.015713,2


In [28]:
X_test = torch.tensor(test_df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]].values, dtype=torch.float32)
X_test

tensor([[3.0000, 1.0000, 0.4282,  ..., 0.0000, 0.0153, 1.0000],
        [3.0000, 0.0000, 0.5853,  ..., 0.0000, 0.0137, 2.0000],
        [2.0000, 1.0000, 0.7738,  ..., 0.0000, 0.0189, 1.0000],
        ...,
        [3.0000, 1.0000, 0.4785,  ..., 0.0000, 0.0142, 2.0000],
        [3.0000, 1.0000,    nan,  ..., 0.0000, 0.0157, 2.0000],
        [3.0000, 1.0000,    nan,  ..., 1.0000, 0.0436, 0.0000]])

In [31]:
with torch.no_grad():
    y_predicted = clf(X_test)

In [33]:
y_predicted.argmax(axis=1)

tensor([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,